# Notebook Extra

## Dependências Funcionais e Normalização

Na primeira parte do curso, vimos como podemos utilizar o modelo relacional, o modelo entidade-relacionamento e DDL (*Data Definition Language*) para especificar uma base de dados, desde a modelagem até sua implementação.

Nos exercícios propostos, percebemos que existem diversas maneiras diferentes de modelar uma base de dados. Será que existem maneiras de avaliarmos qual a melhor solução? Ou ainda, existem padrões a serem seguidos, de forma a facilitar a produção de modelos mais adequados? Nesta aula, pretendemos responder estas perguntas.

## Dependências Funcionais

As **dependências funcionais** são um conceito fundamental em bancos de dados relacionais. Elas descrevem a **relação entre as colunas** de **uma tabela**, especificando quais colunas são **determinadas** por outras colunas.

Compreender as dependências funcionais é importante porque permite que os desenvolvedores de banco de dados projetem esquemas **eficientes** e **confiáveis**, garantindo a **integridade** dos dados armazenados em uma tabela. Quando as dependências funcionais são bem definidas, é possível evitar a **redundância** de dados ou **inconsistências** e garantir que as atualizações feitas em uma coluna sejam propagadas corretamente para todas as outras colunas dependentes.

Além disso, uma compreensão adequada deste tópico irá fornecer base para o entendimento de **normalização**, tópico principal desta aula.

### Definição Formal

Dependências funcionais são relacionamentos do tipo muitos para um. Para dois conjuntos de atributos $𝑋$ e $𝑌$ de uma relação, dizemos que **$𝑌$ é funcionalmente dependente de $𝑋$** ou que **$𝑋$ determina funcionalmente $𝑌$** (e denotamos $𝑋→𝑌$) se, e somente se, qualquer tupla em $𝑋$ tem precisamente uma tupla equivalente em $𝑌$.

Ou seja:

$𝑋→𝑌⇔$ duas linhas da tabela que concordam em $𝑋$ deverão concordar em $𝑌$.

### Exemplo

Seja o *schema*:

`FilmesAtores(id_filme, filme, estúdio, data, id_ator, ator, salario)`

Considere que um mesmo estúdio jamais lança filmes com o mesmo título na mesma data.

Use as abreviações a seguir:

| Atributo | Abreviação |
|----------|:----------:|
| id_filme |     #F     |
| filme    |      F     |
| estúdio  |      E     |
| data     |      D     |
| id_ator  |     #A     |
| ator     |      A     |
| salário  |      S     |

As dependências funcionais mais importantes neste problema são:
- $\{\#𝐹\} → \{𝐹, 𝐸, 𝐷\}$
- $\{\#𝐴\} → \{𝐴\}$
- $\{\#𝐹, \#𝐴\} → \{𝑆\}$
- $\{𝐹,𝐸,𝐷\}→\{\#𝐹\}$

Porém existem várias outras dependências funcionais que podem ser **derivadas** destas através da aplicação de algumas regras:

1. **Reflexão:** $𝐵⊂𝐴⇒𝐴→𝐵$

1. **Aumento:** $𝐴→𝐵⇒𝐴𝐶→𝐵𝐶$

1. **Transitividade:** $(𝐴→𝐵) ∧ (𝐵→𝐶) ⇒𝐴→𝐶$

Algumas dependências funcionais resultantes da aplicação destas regras:

1. **Reflexão:** $\{F\}⊂\{F, S, D\}⇒\{F, S, D\}→\{F\}$

1. **Aumento:** $\{\#F\}→\{F\}⇒\{\#F, A\}→\{F, A\}$

1. **Transitividade:** Considere que $I$ representa as iniciais do ator. Então $(\{\#A\}→\{A\}) ∧ (\{A\}→\{I\}) ⇒ \{\#A\}→\{I\}$

### Fecho de um conjunto de atributos

Para uma relação $𝑟$, um conjunto $𝑆$ de dependências funcionais e um conjunto $𝑍$ de atributos, o **fecho** $𝑍^+$  de $𝑍$ é o conjunto **todos os atributos** da relação $𝑟$ que são **funcionalmente dependentes** de $𝑍$. Para obter $𝑍^+$ podemos aplicar o seguinte algoritmo:

1. algoritmo FECHO ($𝑍$, $𝑆$):
    1. $𝑍^+$ = $𝑍$ (por reflexão)
    1. mudou = Verdadeiro
    1. enquanto mudou:
        1. mudou = Falso
        1. para $𝑋→𝑌$ em 𝑆:
            1. se  $(𝑋⊂𝑍^+) ∧ (𝑌⊄𝑍^+ )$:
                1. $𝑍^+  = 𝑍^+  ∪ 𝑌$  (por transitividade) 
                1. mudou=Verdadeiro
    1. retorna  $𝑍^+$


Como exemplo, o fecho de $Z = \{\#F\}$ com as regras $S = \{ \{\#F\}→\{F\}, \{\#F\}→\{E\}, \{\#F\}→\{D\}, \{\#A\}→\{A\}, \{\#F\#A\}→\{S\}, \{F, E, D\}→\{\#F\} \}$

será

$Z^+ = \{\#F, F, E, D\}$

### Chaves

Vimos nas aulas anteriores a importância de garantir a identificação exclusiva dos elementos de uma tabela. Nas dependências funcionais, vamos expandir nossa definição de chave com:

**Superchave:** um conjunto de atributos $𝐾$ é uma superchave da relação se $𝐾^+$  é o conjunto completo de atributos desta relação. Em outras palavras, superchave é um conjunto de um ou mais atributos que podem identificar **exclusivamente** cada linha em uma tabela de banco de dados. 

**Chave candidata:** uma superchave irredutível, que não contém subconjuntos menores que também são superchaves. Ou seja, nenhum atributo pode ser removido da superchave sem que seu status de superchave seja cancelado.

**Chave primária:** a chave candidata escolhida como identificador da relação por quem está desenhando o banco de dados.


## Formas Normais

Inventadas por Edgar F. Codd, criador do modelo relacional, em 1970. São regras para ajudar a diminuir **redundância** e aumentar **integridade** no projeto de um banco de dados.

Chamamos de **normalização** o processo de rearranjar o banco de dados para que obedeça às várias formas normais. O processo contrário, de regredir o projeto do banco de dados para formas normais menores, chama-se **denormalização**.

#### `1NF`: Primeira forma normal (first normal form)

Uma entidade está na primeira forma normal quando cada tupla tem apenas um valor por atributo. Ou seja, cada célula da tabela é simples e possui apenas um valor.

Considere o seguinte exemplo:

| id_usuário (PK) | nome    | id_endereço | endereço                                   | telefone           |
|-----------------|---------|-------------|--------------------------------------------|--------------------|
| 1               | Juca    | 100         | Rua Quatá, 152, São Paulo/SP               | 111-1111, 333-3333 |
| 2               | Maria   | 200         | Rua Casa do Ator, 537, Ap 42, São Paulo/SP | 222-2222           |
| 3               | Roberto | 300         | Av. Bias Fortes 382, Belo Horizonte/MG     | 444-4444           |

**Pergunta**: Em quais colunas temos múltiplos valores por célula?

<div class="alert alert-success">

Sua resposta AQUI!

</div>

<a href="#" title="Temos múltiplos valores por célula tanto em endereço (struct) quanto em telefone (array)!">Pare o  mouse aqui para ver a resposta</a>

Ao analisar a tabela, podemos perceber que ela não está na **1NF**. Como solução, podemos:
- Criar novas colunas separando endereço, cidade, estado, etc.
- Separar os múltiplos telefones em linhas (tuplas) diferentes

**SOLUÇÃO:**
| id_usuário (PK) | nome    | id_endereço | logradouro            | número | cidade         | estado | telefone |
|-----------------|---------|-------------|-----------------------|--------|----------------|--------|----------|
| 1               | Juca    | 100         | Rua Quatá             | 152    | São Paulo      | SP     | 111-1111 |
| 1               | Juca    | 100         | Rua Quatá             | 152    | São Paulo      | SP     | 333-3333 |
| 2               | Maria   | 200         | Rua Casa do Ator      | 537    | São Paulo      | SP     | 222-2222 |
| 3               | Roberto | 300         | Av. Bias Fortes       | 382    | Belo Horizonte | MG     | 444-4444 |

Perceba que introduzimos uma situação de redundância de dados para conseguir atender à **1NF**. Isto poderá gerar algumas **anomalias**:

- **Atualização (update)**: quando a atualização da tabela pode gerar inconsistências em função da redundância de informação.

- **Inserção (insert)**: quando a inserção de dados não pode ser realizada sempre, pelo risco de deixar a base inconsistente.

- **Remoção (delete)**: quando a remoção de linhas da tabela acaba removendo informação totalmente não-relacionada.


Como exemplo de anomalia de atualização, considere uma alteração no endereço do Juca:

| id_usuário (PK) | nome    | id_endereço | logradouro            | número | cidade         | estado | telefone |
|-----------------|---------|-------------|-----------------------|--------|----------------|--------|----------|
| 1               | Juca    | 100         | Rua Quatá             | 152    | São Paulo      | SP     | 111-1111 |
| 1               | Juca    | 100         | Rua Quatá             | **<span style="color:blue">1000</span>**   | São Paulo      | SP     | 333-3333 |
| 2               | Maria   | 200         | Rua Casa do Ator      | 537    | São Paulo      | SP     | 222-2222 |
| 3               | Roberto | 300         | Av. Bias Fortes       | 382    | Belo Horizonte | MG     | 444-4444 |

Uma vez que não foram atualizadas todas as linhas com o endereço de Juca, ficou complicado dizer onde ele mora (número 152 *ou* 1000?!).

Como exemplo de anomalia de inserção, pense se é possível inserir uma cidade sem ninguém residindo na mesma?!

| id_usuário (PK) | nome    | id_endereço | logradouro            | número | cidade         | estado | telefone |
|-----------------|---------|-------------|-----------------------|--------|----------------|--------|----------|
| 1               | Juca    | 100         | Rua Quatá             | 152    | São Paulo      | SP     | 111-1111 |
| 1               | Juca    | 100         | Rua Quatá             | 152    | São Paulo      | SP     | 333-3333 |
| 2               | Maria   | 200         | Rua Casa do Ator      | 537    | São Paulo      | SP     | 222-2222 |
| 3               | Roberto | 300         | Av. Bias Fortes       | 382    | Belo Horizonte | MG     | 444-4444 |
| **<span style="color:red">???</span>** | **<span style="color:red">???</span>** | **<span style="color:red">???</span>** | **<span style="color:red">???</span>** | **<span style="color:red">???</span>** | **<span style="color:blue">Rio Branco</span>** | **<span style="color:blue">AC</span>** | **<span style="color:red">???</span>** |

Já para a anomalia de remoção, podemos pensar em como seria possível manter a cidade de Belo Horizonte na base quando o usuário Roberto for removido, pois ao remover Roberto a cidade desaparece da base!

| id_usuário (PK) | nome    | id_endereço | logradouro            | número | cidade         | estado | telefone |
|-----------------|---------|-------------|-----------------------|--------|----------------|--------|----------|
| 1               | Juca    | 100         | Rua Quatá             | 152    | São Paulo      | SP     | 111-1111 |
| 1               | Juca    | 100         | Rua Quatá             | 1000   | São Paulo      | SP     | 333-3333 |
| 2               | Maria   | 200         | Rua Casa do Ator      | 537    | São Paulo      | SP     | 222-2222 |
| ~~3~~               | ~~Roberto~~ | ~~300~~         | ~~Av. Bias Fortes~~       | ~~382~~    | ~~**<span style="color:red">Belo Horizonte</span>**~~ | ~~**<span style="color:red">MG</span>**~~     | ~~444-4444~~ |

$\{\text{id\_pessoa}, \text{id\_projeto}\} \rightarrow \{\text{doação}\}$

#### `2NF`: Segunda forma normal

Uma entidade está na segunda forma normal quando obedece à **1NF** e todos os seus atributos não-chave dependem funcionalmente da chave primária inteira.

Considere o seguinte exemplo que modela uma base de projetos e doadores.

| id_pessoa (PK) | id_projeto (PK) | nome_pessoa        | nome_projeto  | doação        |
|----------------|-----------------|--------------------|---------------|---------------|
| 1              | 33              | Mario              | Jacaré        | 150           |
| 1              | 44              | Mario              | Sabiá         | 240           |
| 3              | 33              | Ana                | Jacaré        | 350           |

Perceba que a **chave primária** é composta por (`id_pessoa`, `id_projeto`). Entretanto, ao analisar a tabela, vimos que o nome da pessoa depende unicamente de `id_pessoa` e que o `nome_projeto` depende unicamente do `id_projeto`.

A tabela está na **1NF**? **R: Sim!** Não temos arrays, tudo atômico!

A tabela está na **2NF**? **R: Não!** Pelas dependências funcionais, vemos que $\{\text{id\_pessoa}, \text{id\_projeto}\} \rightarrow \{\text{doação}\}$, o que está ok! Entretanto, a dependência funcional $\{\text{id\_pessoa}\} → \{\text{nome\_pessoa}\}$ é um exemplo de dependência funcional que quebra os requisitos da **2NF**.

Para solucionar o problema, fazendo com que a base de dados obedeça à **2NF**, iremos separar a base em três tabelas diferentes:

**Pessoa**:
| id_pessoa (PK) | nome_pessoa |
|----------------|-------------|
| 1              | Mario       |
| 3              | Ana         |

**Projeto**:
| id_projeto (PK) | nome_projeto |
|-----------------|--------------|
| 33              | Jacaré       |
| 44              | Sabiá        |

**doacao**:
| id_pessoa (PK) | id_projeto (PK) | doação |
|----------------|-----------------|--------|
| 1              | 33              | 150    |
| 1              | 44              | 240    |
| 3              | 33              | 350    |

**Dicas**:
- Para estar na **2NF**, no mínimo precisa estar também na **1NF**
- Se está na **1NF** e tem chave primária simples (uma coluna), então já está na **2NF**
- Se está na **1NF** e tem chave primária composta:
    - Todas as colunas que não são chave primária dependem da chave primária **por inteiro**?
        - Se sim, está na **2NF**
        - Se não, não está na **2NF**

#### `3NF`: Terceira forma normal

Uma tabela está na **3FN** quando estiver na **2FN** e se todos os atributos que não pertencem à chave primária não dependerem funcionalmente de outro atributo que não a chave (não existem **dependências transitivas** para a chave primária).

Considere a base de dados:

| id_usuário (PK) | nome    | id_endereço | endereço              | cidade         | estado |
|-----------------|---------|-------------|-----------------------|----------------|--------|
| 1               | Juca    | 100         | Rua Quatá, 300        | São Paulo      | SP     |
| 2               | Maria   | 200         | Rua Casa do Ator, 537 | São Paulo      | SP     |
| 3               | Roberto | 300         | Av. Bias Fortes 382   | Belo Horizonte | MG     |

A tabela está na **1NF**? **R: Sim!**

A tabela está na **2NF**? **R: Sim!**

A tabela está na **3NF**? **R: Não!**

Pelas dependências funcionais, percebemos que $\{\text{id\_usuário}\} → \{\text{id\_endereço}\}$ e que $\{\text{id\_endereço}\} \rightarrow \{\text{endereço}\}$. Logo, temos uma dependência transitiva.

Os atributos de `endereço`, `cidade` e `estado` pertencem ao **endereço** e não diretamente ao **usuário**.

Para solucionar, iremos novamente separar em tabelas diferentes:

**usuário**:
| id_usuário (PK) | nome    | id_endereço (FK) |
|-----------------|---------|------------------|
| 1               | Juca    | 100              |
| 2               | Maria   | 200              |
| 3               | Roberto | 300              |

**endereço**:
| id_endereço (PK) | endereço              | cidade         | estado |
|------------------|-----------------------|----------------|--------|
| 100              | Rua Quatá, 300        | São Paulo      | SP     |
| 200              | Rua Casa do Ator, 537 | São Paulo      | SP     |
| 300              | Av. Bias Fortes 382   | Belo Horizonte | MG     |

Ainda, podemos separar as cidades, deixando a base com três tabelas:

**usuário**:
| id_usuário (PK) | nome    | id_endereço (FK) |
|-----------------|---------|------------------|
| 1               | Juca    | 100              |
| 2               | Maria   | 200              |
| 3               | Roberto | 300              |

**endereço**:
| id_endereço (PK) | endereço              | id_cidade (FK) |
|------------------|-----------------------|----------------|
| 100              | Rua Quatá, 300        | 55             |
| 200              | Rua Casa do Ator, 537 | 55             |
| 300              | Av. Bias Fortes 382   | 66             |

**cidade**:
| id_cidade (PK) | cidade              | estado |
|----------------|---------------------|--------|
| 55             | São Paulo           | SP     |
| 66             | Belo Horizonte      | MG     |
| 300            | Av. Bias Fortes 382 | 66     |

Resumindo:
- Na **1NF** garantimos um valor por célula
- Na **2NF** removemos dependências parciais de chave
- Na **3NF** removemos dependências transitivas

No design de bancos de dados, geralmente é buscado obedecer até a 3NF, entretanto, existem outras formas normais que tratam casos particulares!

## Exercícios

**Exercício 1**: Suponha que tenhamos uma relação $𝑅(𝐴, 𝐵, 𝐶, 𝐷, 𝐸, 𝐹)$ e as seguintes dependências funcionais:

$𝑆= {𝐴→𝐵𝐶, 𝐸→𝐶𝐹, 𝐵→𝐸, 𝐶𝐷→𝐸𝐹}$

(abreviamos $𝐴$ no lugar de $\{𝐴\}$, $𝐵𝐶$ no lugar de $\{𝐵, 𝐶\}$, etc.)


Calcule o fecho $𝑍^+$  de $𝑍={𝐴, 𝐵}$

<a href="#gab_ex1">Click para ver a resposta</a>

**Exercício 2**: Considere o schema do início da aula:

`FilmesAtores(id_filme, filme, estúdio, data, id_ator, ator, salario)`

Normalize esse schema até a 3NF.

<div class="alert alert-success">

Sua resposta AQUI! Crie múltiplas células, conforme necessário!

</div>

<a href="#gab_ex2">Click para ver a resposta</a>

**Exercício 3**: Crie exemplos de relação, para cada caso, explicitando as dependências funcionais:

**a)** Não é 1NF.

<div class="alert alert-success">

Sua resposta AQUI! Crie múltiplas células, conforme necessário!

</div>

**b)** 1NF mas não 2NF.

<div class="alert alert-success">

Sua resposta AQUI! Crie múltiplas células, conforme necessário!

</div>

**c)** 2NF mas não 3NF.

<div class="alert alert-success">

Sua resposta AQUI! Crie múltiplas células, conforme necessário!

</div>

**d)** 3NF

<div class="alert alert-success">

Sua resposta AQUI! Crie múltiplas células, conforme necessário!

</div>

<a href="#gab_ex3">Click para ver a resposta</a>

**Exercício 4**: Considere os dados:

| id_tecnico | nome             | cidade_onde_atende | bairros_onde_atende                | whatsapp                                    |
|------------|------------------|--------------------|------------------------------------|---------------------------------------------|
| 1          | Ana Chavier      | São Paulo, SP      | Centro, Butanta                    | 11 99989-1010, 11 99918-1312                |
| 1          | Ana Chavier      | Campinas, SP       | Bosque, Guanabara, Vila Industrial | 11 99989-1010, 11 99918-1312                |
| 3          | Gabriela Antunes | Brasilia, DF       | Samambaia                          | 61 97899-0051, 61 97899-0052, 61 97899-0053 |
| 4          | Bruno Pereira    | Rio de Janeiro, RJ | Lapa, Ipanema                      | 21 87767-1991                               |

Normalize esse schema até a 3NF.

<div class="alert alert-success">

Sua resposta AQUI! Crie múltiplas células, conforme necessário!

</div>

**Exercício 5**: Você recebeu um arquivo CSV com as informações do Clube dos Jogadores de Bolinha de Gude.

Nota: uma bolinha é identificável pela sua cor, tamanho e material.

| Cor      | Tamanho | RGB         | Peso (g) | Diâmetro  (cm) | Material | Valor  (R$) | Dono | Qtde |
|----------|---------|-------------|----------|----------------|----------|-------------|------|------|
| Azul     | P       | (0, 0, 255) | 2        | 2              | Vidro    | 3.00        | Juca | 3    |
| Vermelha | G       | (255, 0, 0) | 15       | 5              | Vidro    | 10.00       | Ana  | 5    |
| Azul     | G       | (0, 0, 255) | 100      | 5              | Aço      | 15.00       | Juca | 6    |
| Vermelha | P       | (255,0,0)   | 2        | 2              | Vidro    | 4.00        | Ana  | 2    |

**a)** Defina as dependências funcionais essenciais

<div class="alert alert-success">

Sua resposta AQUI! Crie múltiplas células, conforme necessário!

</div>

**b)** Passe para a 3NF

<div class="alert alert-success">

Sua resposta AQUI! Crie múltiplas células, conforme necessário!

</div>

**Exercício 6**: Em certos cenários, bases são desenhadas de forma que as regras de normalização de dados são intencionalmente violadas.

**a)** Você consegue pensar em algum cenário onde seria recomendado desobedecer alguma das formas normais (ou todas)?

<div class="alert alert-success">

Sua resposta AQUI!

</div>

**b)** Vamos buscar expandir nossos conceitos! Pesquise no Google e no ChatGPT sobre **desnormalização** de banco de dados. Escreva abaixo suas conclusões.

<div class="alert alert-success">

Sua resposta AQUI!

</div>

## Referências
- Silberschatz, A., Korth, H. F., Sudarshan, S. “Sistema de Banco de Dados”, CAP 8.
- Date, C. J. “Introdução a Sistemas de Bancos de Dados”. CAPs 11 a 13.
- https://chatbot.theb.ai/

## Gabarito


**<div id="gab_ex1">Exercício 1</div>**
<div class="alert alert-warning">

$Z^+ = \{A, B, C, E, F\}$
</div>

**<div id="gab_ex2">Exercício 2</div>**
<div class="alert alert-warning">

Considerando as abreviações

| Atributo | Abreviação |
|----------|:----------:|
| id_filme |     #F     |
| filme    |      F     |
| estúdio  |      E     |
| data     |      D     |
| id_ator  |     #A     |
| ator     |      A     |
| salário  |      S     |

Construímos:

    - Filmes (#F, F, E, D)
    - Atores (#A, A)
    - Salarios (#F, #A, S)
    
Está na 3NF porque:
- Cada célula é simples (1NF)
- Não tem dependência parcial da PK (2NF)
- Não tem dependência transitiva (3NF)

Dica: construa as relações funcionais!
</div>

**<div id="gab_ex3">Exercício 3</div>**
<div class="alert alert-warning">

**a**) Deixo duas respostas, dentre infinitas possíveis:
- `Pessoa(nome, endereço)` onde endereço é uma célula composta, como uma lista
- `Filme(id_filme, filme, tags)` onde tags é uma célula composta
    

**b)**
`R(A [PK], B [PK], C, D)`

$AB → C$
$A → D$


**c)**
`R(A [PK], B)`

$A → B$
    
</div>